In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('train.csv')

In [3]:
import math
def deg_to_rad(dr):
    return (dr*math.pi)/180

df['pick_lat'] = deg_to_rad(df['pick_lat'])
df['pick_lon'] = deg_to_rad(df['pick_lon'])
df['drop_lat'] = deg_to_rad(df['drop_lat'])
df['drop_lon'] = deg_to_rad(df['drop_lon'])

In [4]:
dlon = df['drop_lon'] - df['pick_lon']
dlat = df['drop_lat'] - df['pick_lat']

In [5]:
df['dist'] = np.sin(dlat/2)**2 + np.cos(df['pick_lat'])*np.cos(df['drop_lat'])*np.sin(dlon/2)**2

In [6]:
df['dist'] = 2*np.arctan2(df['dist']**0.5,(1-df['dist'])**0.5)

In [7]:
R = 6373.0
df['dist'] = R*df['dist']

In [8]:
df['fare'].count()
df['label'] = df['label'].map({'correct': 1, 'incorrect': 0})

In [9]:
df = df.drop(['pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon'], axis = 1) 

In [10]:
#drop na
df = df.dropna(how='any')           # assign back
df.dropna(how='any', inplace=True)

In [11]:
y = df['label']
df = df.drop(['label'], axis = 1)

In [12]:
X = df

In [38]:
from sklearn.model_selection import train_test_split
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [39]:
import xgboost as xgb

D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)

In [40]:
param = {
    'max_depth': 100,  
    'objective': 'multi:softprob',  
    'num_class': 2} 

steps = 30  # The number of training iterations

In [41]:
model = xgb.train(param, D_train, steps)

In [42]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))
print("F1 score = {}".format(f1_score(y_test, best_preds)))

Precision = 0.9067639217565826
Recall = 0.7801315640769436
Accuracy = 0.950108033785111
F1 score = 0.9729095563139932


In [46]:
from xgboost import XGBClassifier

In [47]:
model = XGBClassifier()
 
# fit the model with the training data
model.fit(X_train,y_train)
 
 
# predict the target on the train dataset
predict_train = model.predict(X_train)
print('\nTarget on train data',predict_train) 
 
# Accuray Score on train dataset
accuracy_train = accuracy_score(y_train,predict_train)
print('\naccuracy_score on train dataset : ', accuracy_train)
 
# predict the target on the test dataset
predict_test = model.predict(X_test)
print('\nTarget on test data',predict_test) 
 
# Accuracy Score on test dataset
accuracy_test = accuracy_score(y_test,predict_test)
print('\naccuracy_score on test dataset : ', accuracy_test)


Target on train data [0 1 1 ... 1 1 1]

accuracy_score on train dataset :  0.9906542056074766

Target on test data [1 1 1 ... 1 1 1]

accuracy_score on test dataset :  0.9485366332744058


In [48]:
print("F1 score = {}".format(f1_score(y_test, predict_test)))

F1 score = 0.9720503520375507
